# Contexte

Ce notebook tente de résoudre le bug 245 :
https://github.com/dataforgoodfr/13_potentiel_solaire/issues/245

Pour rappel, les livrables sont :

* Estimer le nb / % d'établissements scolaires pour lesquels coordonnées & adresses semblent incohérents
* Script python avec la logique permettant l'amélioration de la qualité des données de l'annuaire
* Optionnel : migration alembic pour mettre à jour la base de données potentiel_solaire.duckdb (@machbry s'en occupera si besoin)


# Données

## Imports et fonctions d'appoint

Nous utilisons deux jeux de données :
1. celui contenant la géométrie (empreinte au sol) des bâtiments, provenant des zones d'éducation (un fichiers gpkg par département, prétraité par notre algorithme) ; il notamment contient l'identifiant de l'établissement, et un MULTIPOLYGON.
2. celui de l'annuaire des établissements scolaires, qui contient le même identifiant, une adresse (code postal, région, département...), et des coordonnées sous forme de POINT.

Après les imports nous définissons trois fonction d'appoint, respectivement pour créer une adresse en concaténant les colonnes correspondantes, d'une fonction de géocodage (utilisant l'excellent service de l'IGN), et d'un calcul de distance entre deux objets shapely, par défaut dans l'EPSG 6933 comme défini ici : https://outline.services.dataforgood.fr/doc/choix-du-crs-pour-les-calculs-de-distance-fFPOfhd1mi




In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely import Point


import requests


pd.options.display.max_rows=200


lamb93="EPSG:2154"
wgs84="EPSG:4326"
projglob="EPSG:6933"

def cree_adresse(df: 'pd.DataFrame', cols: list[str] = None) -> 'pd.Series':
    """
    Construit une adresse complète à partir des colonnes spécifiées d'un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les colonnes d'adresse.
        cols (list[str], optional): Liste des noms de colonnes à utiliser pour l'adresse.
            Par défaut ["adresse_1", "code_postal", "nom_commune"].

    Returns:
        pd.Series: Série contenant les adresses concaténées.
    """
    if cols is None:
        cols = ["adresse_1", "code_postal", "nom_commune"]
    return df[cols].fillna(" ").apply(lambda x: ' '.join(x.values), axis=1)


def geocode(adresse: str) -> 'Point | None':
    """
    Géocode une adresse en utilisant l'API IGN (data.geopf.fr).

    Args:
        adresse (str): L'adresse à géocoder, typiquement obtenue via cree_adresse().

    Returns:
        Point | None: Un objet Point avec les coordonnées géographiques (WGS84) ou None si échec.
    """
    #time.sleep(0.1)
    r = requests.get(url="https://data.geopf.fr/geocodage/search",
                     params={"q": adresse})
    if not r.ok:
        return None
    try:
        js = r.json()
        return Point(js['features'][0]['geometry']['coordinates'])
    except:
        return None

def calcul_dist(df: 'pd.DataFrame', col1: str, col2: str, crs: str = projglob) -> 'pd.Series':
    """
    Calcule la distance entre deux colonnes de géométrie dans un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les colonnes de géométrie.
        col1 (str): Nom de la première colonne.
        col2 (str): Nom de la seconde colonne.
        crs (str, optional): Système de référence de coordonnées cible (projetées)
                             Par défaut EPSG:6933 (utilisée par le projet).

    Returns:
        pd.Series: Série contenant les distances calculées.
    """
    return gpd.GeoSeries(df[col1], crs="WGS84").to_crs(crs)\
        .distance(gpd.GeoSeries(df[col2], crs="WGS84").to_crs(crs))



## Nous commençons par les zones éducatives

On peut se contenter dans cette étude d'idcol (commune à l'annuaire), toponyme et geometry. 

La séparation en multipolygones rend possible, pour un même site, la présence de nombreux doublons. Nous en comptons 2200, et ne les considérerons pas à notre niveau d'analyse.

In [ ]:
folder_path='../data/results/'
files = glob.glob(f"{folder_path}/D*.gpkg")
gdfs = [gpd.read_file(f, layer="educational_zones") for f in files]
gezraw=gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)
gezraw

In [ ]:
idcol='identifiant_de_l_etablissement'

gez=gezraw[[idcol, "toponyme", "geometry"]]
gez

In [ ]:
gez[gez[idcol].duplicated(keep=False)]

In [ ]:
gez=gez.drop_duplicates(subset=idcol)
gez

## Annuaire des établissements

Nous poursuivons avec l'annuaire.

Nous avons cette fois 7 paires de doublons ; ils ont même adresse et geometry, dans le contexte de cette étude nous pouvons les supprimer.

Une fois les doublons supprimés, nous créons un dataframe dfm fusionné sur la colonne idcol.



In [ ]:
goodcols=[idcol, 'nom_etablissement','adresse_1', 'code_postal', 'nom_commune',
          'libelle_departement',  'libelle_region', 'geometry']
gann=gpd.read_file("../data/etablissements.geojson", columns=goodcols).sort_values(idcol)
gann

In [ ]:
gann.dropna(subset="geometry", inplace=True)
gann

In [ ]:
gann[gann.duplicated(subset=[idcol], keep=False)]

In [ ]:
dfm=gann.merge(gez.rename(columns={"geometry": "polygon"}), on=idcol)
dfm["dist"]=calcul_dist(dfm, "geometry", "polygon")
dfm.sort_values("dist", inplace=True, ascending=False)
dfm["adresse"]=cree_adresse(dfm)
dfm

# Statistiques des écarts

L'on cherche à savoir s'il y a une coupure nette des distances entre une identification correcte et une qui ne l'est pas.

Pour ce faire, nous regardons les distributions associées (classique et cumulée)


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
bins=np.logspace(-3, 5, 81)
plt.hist(dfm.dist, bins=bins)
plt.xlabel("Distance entre les coordonnées GPS du bâtiment physique et de l'adresse (m)")
plt.ylabel("Nombre d'établissements concernés")
plt.title("Distribution de la distance des empreintes aux coordonnées (échelle log/log)")
ax.loglog()
ax.grid()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
plt.hist(dfm.dist, bins=bins, cumulative=True, density=True)
plt.xlabel("Distance entre les coordonnées GPS du bâtiment physique et de l'adresse (m)")
plt.ylabel("Nombre d'établissements concernés")
plt.title("Distribution cumulée de la distance des empreintes aux coordonnées")
ax.grid()
ax.set_xscale('log')


In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
plt.hist(dfm.dist, bins=bins, cumulative=True, density=True)
plt.xlabel("Distance entre les coordonnées GPS du bâtiment physique et de l'adresse (m)")
plt.ylabel("Nombre d'établissements concernés")
plt.title("Distribution cumulée de la distance des empreintes aux coordonnées")
ax.grid()
ax.set_xscale('log')
ax.set_ylim(0.8, 1)
ax.set_xlim(100, 1e5)

On observe une distribution monomodale des distances, centrée à quelques dizaines de mètres.

Les distributions cumulées confirment ce diagnostic : environ 50% des établissements à une distance inférieure à une vingtaine de mètres, 80% à 100 mètres, 920% à 250m, 2.5 % à 1 km, et peut-être un millième au-delà de 10 km.

Il faut fixer un seuil au-delà duquel on estimera qu'il y a un problème. Une solution devra alors être envisagée. Nous suggérons 300m, ce qui correspond tout de même près de 10 % des établissements.

Nous regardons s'il existe des différences au sein des régions et départements ; nous trions par distance au troisième quartile (le pire quart des établissements au-dessus de cette valeur).


In [ ]:
dfm.groupby("libelle_region")["dist"].describe().sort_values("75%", ascending=False)

In [ ]:
dfm.groupby("libelle_departement")["dist"].describe().sort_values("75%", ascending=False)

Les DROM, en particulier la Guadeloupe (50% supérieurs à 200 m), se distinguent par des écarts élevés, mais aucune région n'est épargnée et toutes ont des établissements plus loin qu'un kilomètre de leurs coordonnées.

Après confirmation sur osm, les bâtiments de quelques établissements sont bien placés, mais ce sont les coordonnées GPS qui ne correspondent pas à la réalité.

# Exemple de la git issue et géocodage

Nous reprenons l'exemple du lycée Germaine Tillion.

Nous observons bien l'écart constaté d'un kilomètre.

En géocodant l'adresse, on obtient des coordonnées plus proches de celles du polygone.

Est-ce qu'un géocodage des établissements où la distance est la plus grande serait la solution ?

In [ ]:
idgermaine='0110012D'
dfgermaine=dfm.query(f"{idcol}=='{idgermaine}'").iloc[0].copy()
dfgermaine

In [ ]:
print()

In [ ]:
dfgermaine["geocode"]=geocode(dfgermaine.adresse)

In [ ]:
calcul_dist(dfgermaine, "polygon", "geocode")

En l'espèce, cela se passe beaucoup mieux : géocoder de nouveau l'adresse permet de la faire coller à la position des bâtiments.

Pour savoir si cela généralise, nous sélectionnons au hasard dix établissements dans chaque département, pour avoir une variété ; les biais d'échantillonnage seraient intéressants à discuter, mais ça sera pour une autre fois ;)



In [ ]:
dfsamp=dfm.groupby("libelle_departement").sample(10).copy()

In [ ]:
geocodes=dfsamp.adresse.apply(geocode)
geocodes

In [ ]:
dfsamp["geocodes"]=geocodes
dfsamp["dist2"]=calcul_dist(dfsamp, "polygon", "geocodes")

dfsamp

In [ ]:
import colorcet as cc
cmap=cc.glasbey
dfsamp["dpt"]=dfsamp.code_postal.astype(int)//1000
dfsamp["color"]=dfsamp["dpt"].apply(lambda x: cmap[x])
dfsamp

In [ ]:
import matplotlib.pyplot as plt
import colorcet as color

fig, ax=plt.subplots(figsize=(10, 10))
plt.scatter(dfsamp.dist, dfsamp.dist2, color=dfsamp.color, alpha=0.8)
p1, p2=(1e-2, 1e6)
plt.axline((p1, p1), (p2, p2) , color="k")

ax.loglog()
ax.set_xlabel("Distance originale")
ax.set_ylabel("Distance après géocode")
ax.set_xlim(p1, p2)
ax.set_ylim(p1, p2)
ax.grid()

# Analyse et recommandation

On observe un bel effet de régression à la moyenne : les points initialement loin sont statistiquement plus proches après géocodage (les points sous la droite ont une abscisse élevée), et les points initialement proches sont plus distants (les points d'abscisse faible sont au-dessus de la courbe).

La décision n'est pas évidente. On pourrait, pour les établissements initialement distants, voir si un géocodage réduit la distance, auquel cas prendre cette nouvelle position pour la carte.

Ou bien partir du centroïde des positions de l'empreinte des bâtiments (qui semble systématiquement fiable, empiriquement et par construction) défini comme point indiqué sur la carte. On pourrait envisager un géocoding inverse, mais cela soulèverait sans doute plus de questions en cas d'écart constaté par les utilisateurs à l'adresse réelle que de garder l'adresse connue et d'en déplacer les coordonnées.

Une fonction proposée, utilisant les fonctions d'appoint déjà définies, et partant d'un dataframe mergé, pourrait avoir cette allure :



In [ ]:
def reduction_ecarts(df: 'pd.DataFrame', seuil: int = 300) -> None:
    """
    Réduit les écarts de géolocalisation en optimisant les points selon la distance à un polygone.

    Args:
        df (pd.DataFrame): DataFrame contenant les colonnes 'geometry' et 'polygon'.
        seuil (int, optional): Seuil de distance pour optimisation. Par défaut 300.

    Modifie le DataFrame en ajoutant les colonnes:
        - 'dist': distance entre 'geometry' et 'polygon'
        - 'adresse': adresse concaténée
        - 'optim_geo': géométrie optimisée
        - 'optim_dist': distance entre 'polygon' et 'optim_geo'
        - 'final_pt': point final optimisé
    """
    df["dist"] = calcul_dist(df, "geometry", "polygon")
    df["adresse"] = cree_adresse(df)
    df["optim_geo"] = df.apply(lambda x: x["geometry"] if x["dist"] < seuil else geocode(x["adresse"]), axis=1)
    df["optim_dist"] = calcul_dist(df, "polygon", "optim_geo")
    df["final_pt"] = df.apply(lambda x: x["optim_geo"] if x["optim_dist"] < seuil else x["polygon"].centroid, axis=1)
    return df

dft=dfm.sample(100).copy()
dft=reduction_ecarts(dft)
dft